In [ ]:
import pandas as pd
tweets=pd.read_csv("tweets.csv",encoding="utf-8")
print(tweets.keys())

In [ ]:
tweets=tweets.drop(['source',"id",'in_reply_to_status_id','in_reply_to_user_id'
                    ,"truncated","in_reply_to_screen_name","retweeted_status_id","geo","place","timestamp"],axis=1)
for i in tweets.keys():
    if i not in ['created_at', 'text', 'user_id', 'retweet_count', 'reply_count',
       'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions']:
        tweets=tweets.drop([i],axis=1)
print (tweets.keys())

In [ ]:
created_at=tweets["created_at"]
created_at=created_at.str.split(' ', expand=True)
tweets=tweets.drop(["created_at"],axis=1)

In [ ]:
num_years=len(created_at[5].unique())
print (num_years)

In [ ]:
for i in tweets.keys():
    x=tweets[i]
    print (i,x.dropna().shape)

In [ ]:
reps=tweets["reply_count"]
reps=reps.fillna(0)
tweets["reply_count"]=reps
for i in tweets.keys():
    x=tweets[i]
    print (i,x.dropna().shape)

In [ ]:
tweets=tweets.dropna()
print (tweets.shape)

In [ ]:
def calculate_tweet_len(row):
    return len(row["text"])

def calculate_tweet_link_count(row):
    return row["text"].count("http")
def calculate_tweet_count(row):
    return 1
def label(row):
    return 0
def tweets_frequency(row):
    return row["num_tweets"]/(num_years*365)
def over_50_tweets(row):
    if row["num_tweets"]>50:
        return 1
    else :
        return 0

In [ ]:
tweets["tweet_length"]=tweets.apply(calculate_tweet_len,axis=1)
tweets["link_count"]=tweets.apply(calculate_tweet_link_count,axis=1)
tweets["num_tweets"]=tweets.apply(calculate_tweet_count,axis=1)
print (tweets["num_tweets"])
num = tweets.groupby('user_id').sum().reset_index()
for i in num.keys():
    if i != "user_id" and i!="num_tweets":
        num=num.drop(i,axis=1)
num_tweets = num.groupby('user_id').sum().reset_index()
num_tweets["tweets_frequency"]=num_tweets.apply(tweets_frequency,axis=1)
num_tweets["over_50_tweets"]=num_tweets.apply(over_50_tweets,axis=1)
print (num_tweets)

In [ ]:

tweets=tweets.drop(["text"],axis=1)
print (tweets.shape)

In [ ]:
print (tweets.keys())

In [ ]:
tweets=tweets.apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [ ]:
result = tweets.groupby('user_id').mean().reset_index()
result=pd.merge(result,num_tweets,on="user_id")
result=result.drop("num_tweets_x",axis=1)

In [ ]:
print(result.keys())


In [ ]:
result

In [ ]:
users=pd.read_csv("users.csv")
if "id" in users.keys():
    users=users.rename(columns={"id":"user_id"})
    print("renamed")
print (users.shape)
drop_list=["name","url","lang","created_at","time_zone","location",
           "default_profile_image","profile_image_url","profile_banner_url","profile_use_background_image",
           "profile_background_image_url_https","profile_text_color","profile_image_url_https","profile_sidebar_border_color",
           "profile_background_tile","profile_sidebar_fill_color","profile_background_image_url","profile_background_color",
           "profile_link_color","utc_offset","description","dataset","updated","protected","verified"]
users["default_profile"]=users["default_profile"].fillna(0)
print (len(drop_list))
for i in drop_list:
    try: users=users.drop(i,axis=1)
    except : pass
print (users.shape)

In [ ]:
Final_DataFrame=pd.merge(result,users,on="user_id")
print (Final_DataFrame.shape,Final_DataFrame.keys())


In [ ]:
print (users.keys(),users.shape)
print (result.keys(),result.shape)
print (Final_DataFrame.keys(),Final_DataFrame.shape)

In [ ]:
Final_DataFrame=Final_DataFrame.fillna(0)
print (Final_DataFrame)
Final_DataFrame["label"]=Final_DataFrame.apply(label,axis=1)


In [ ]:
Final_DataFrame.keys()

In [ ]:
cols=['user_id', 'retweet_count', 'reply_count', 'favorite_count',
       'num_hashtags', 'num_urls', 'num_mentions', 'tweet_length',
       'link_count', 'num_tweets_y', 'tweets_frequency', 'over_50_tweets',
       'screen_name', 'statuses_count', 'followers_count', 'friends_count',
       'favourites_count', 'listed_count', 'default_profile', 'geo_enabled',
       'label']
for i in Final_DataFrame.keys():
    if i not in cols:
        Final_DataFrame=Final_DataFrame.drop([i], axis=1)

In [ ]:
print (Final_DataFrame.shape)
Final_DataFrame.keys()

In [ ]:
Final_DataFrame.to_csv("final_dataset.csv",index=False)